### Prompt

Every year as the Oscars roll around, John reviews and organizes his movie collection. He has a multi-tab spreadsheet with information about movies, including:

•	Name  
•	Year of release  
•	Format (DVD, BR, Disc, etc.)  
•	Movie rating  
•	Director  
•	Ownership status (OWN indicates John has a copy, 1 means he intends to purchase)  

The spreadsheet is divided into several tabs: Oscar best picture (BP), best director (BD), best actress, and best actor, among others. There are also "Top Lists" from various sources, including, but not limited to: IMDB, Variety, Time Magazine, and the British Academy of Film and Television Arts (BAFTA).
There is also another spreadsheet that has a list of the sheet names.

As you can imagine, the data is messy. It became too much work for John to maintain the spreadsheet, and he decided to leverage Alteryx to help him organize the data. John also wants to remove the On Line FilmTV Assoc and Warren Miller tabs from his spreadsheet, as he no longer wishes to maintain them.

Your task for this challenge is to create a macro that consolidates all Excel tabs into one spreadsheet. Include these categories:

- Movie title
- Year the movie was released
- Movie rating
- Status (whether John owns the movie or not)
- Original spreadsheet tab it comes from

In [1]:
# import lib

import pandas as pd

import os

import re

In [2]:
# importing sheet names

cwd = os.getcwd()

file_names = ['sheet_names.csv']

file_path = []

for x in file_names:
    
    file_path.append(os.path.join(cwd, x))

dfs = {}

for x in file_path:
    
    file_name = os.path.basename(x)
    
    dfs[file_name] = pd.read_csv(x, encoding = 'ISO-8859-1')

In [3]:
sheet_names = dfs['sheet_names.csv']

In [4]:
# filtering out sheet names

sheet_filtered = sheet_names[~sheet_names['Sheet Names'].isin(['On Line FilmTV Assoc', 'Warren Miller'])]

In [5]:
sheet_filtered.head()

,Sheet Names,FileName,FileName2
0,BP,\\Mac\Home\Library\CloudStorage\OneDrive-alter...,C:\Users\CharlesYi\Downloads\Challenge466_star...
1,Should Be BP,\\Mac\Home\Library\CloudStorage\OneDrive-alter...,C:\Users\CharlesYi\Downloads\Challenge466_star...
2,BD,\\Mac\Home\Library\CloudStorage\OneDrive-alter...,C:\Users\CharlesYi\Downloads\Challenge466_star...
3,Best Actor,\\Mac\Home\Library\CloudStorage\OneDrive-alter...,C:\Users\CharlesYi\Downloads\Challenge466_star...
4,Best Actress,\\Mac\Home\Library\CloudStorage\OneDrive-alter...,C:\Users\CharlesYi\Downloads\Challenge466_star...


In [6]:
# creating list of sheets

sheet_names = sheet_filtered['Sheet Names'].tolist()

In [7]:
# extracting file path for movies

movies_path = r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 466_Organize Oscar Winner Data\Movie Inventory Challenge1.xlsx'

movies_path

'C:\\Users\\CharlesYi\\Jupyter Notebook\\Alteryx Challenges\\Challenge 466_Organize Oscar Winner Data\\Movie Inventory Challenge1.xlsx'

In [8]:
# creating loop to bring in all data in tabs into one df
    
dfs = []  
    
for sheet in sheet_names:
    
    df = pd.read_excel(movies_path, sheet_name = sheet, engine = 'openpyxl')
    
    df['sheet'] = sheet
    
    dfs.append(df)

In [9]:
# combining all dfs

df = pd.concat(dfs, ignore_index = True)

In [10]:
df.head()

,Format,YR,TITLE,DIRECTOR,STATUS,sheet,Won,Actor,Status,Actress,...,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Studio,Rank,Order,RANK,Rating
0,BR,1927,Wings,William A. Wellman,OWN,BP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Disc,1928,The Broadway Melody,Norman Taurog,OWN,BP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Disc,1929,All Quiet on the Western Front,Lewis Milestone,OWN,BP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Disc,1930,Cimarron,Wesley Ruggles,OWN,BP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Disc,1931,Grand Hotel,Edmund Goulding,OWN,BP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# dropping unnamed columns

# filtering to unnamed columns

unnamed_col = df.columns

unnamed_col = [x for x in unnamed_col if 'Unnamed' in x]

# creating function to iterate and drop

for x in unnamed_col:
    
    df = df.drop(columns = x)
    
# Alternative

# df.drop(columns = unnamed_col)

In [13]:
# changing data type for rating 

df['Rating'] = df['Rating'].astype('float64')

In [14]:
# adjusting rating

df['Rating'] = df['Rating'].apply(lambda x: x * 10 )

In [16]:
# coalescing rank and rating

df['Rating'] = df['Rating'].combine_first(df['Rank'])

df.head()

,Format,YR,TITLE,DIRECTOR,STATUS,sheet,Won,Actor,Status,Actress,Inducted,Director(s),Country,Studio,Rank,Order,RANK,Rating
0,BR,1927,Wings,William A. Wellman,OWN,BP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Disc,1928,The Broadway Melody,Norman Taurog,OWN,BP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Disc,1929,All Quiet on the Western Front,Lewis Milestone,OWN,BP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Disc,1930,Cimarron,Wesley Ruggles,OWN,BP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Disc,1931,Grand Hotel,Edmund Goulding,OWN,BP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# filtering to title not null

df = df[~df['TITLE'].isnull()]

df.shape

(2404, 18)

In [37]:
# selecting rel col

df = df[['TITLE', 'YR', 'Rating', 'STATUS', 'sheet']]

# sorting by title

df['TITLE'] = df['TITLE'].astype('str') # converting to string

df['TITLE'] = df['TITLE'].str.strip() # trimming

final_df = df.sort_values(by = ['TITLE', 'YR']).reset_index(drop = True)

In [38]:
final_df

,TITLE,YR,Rating,STATUS,sheet
0,10 Commandments,1956,NaN,OWN,Should Be BP
1,12 Angry Men,1957,NaN,OWN,Lib of Congress
2,12 Angry Men,1957,87.0,OWN,AFI Post 2007 T100
3,12 Angry Men,1957.0,26.0,OWN,Parade T100
4,12 Angry Men,1957,90.0,OWN,IMDB T250
...,...,...,...,...,...
2399,Your Name,2016,84.0,NaN,IMDB T250
2400,Zapruder Film,1963,NaN,NaN,Lib of Congress
2401,Zoot Suit,1981,NaN,NaN,Lib of Congress
2402,"sex, lies, and videotape",1989,NaN,NaN,Lib of Congress
